In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from Bio import SeqIO

In [4]:
import re
import random
import numpy as np
import pandas as pd
from itertools import product
from mofiwo.utility import generate_utr_from_cdna_cds, load_rna_fasta_zipfile
from mofiwo.analysis import generate_k_mer, generate_feature_by_kmer_loc

In [7]:
zip_cdna = r'D:\workspace\rnamotif\samples\s1_cdna.zip'
zip_cds = r'D:\workspace\rnamotif\samples\s1_cds.zip'

In [8]:
dic_cdna = load_rna_fasta_zipfile(zip_cdna)
dic_cds = load_rna_fasta_zipfile(zip_cds)

In [ ]:
dic_utr = generate_utr_from_cdna_cds(dic_cdna, dic_cds)

In [10]:
dic_seq = dict()
for k,v in dic_utr.items():
    dic_seq.update({k: {'CDS': dic_cds[k].seq, 'UT3': v['utr3'], 'UT5': v['utr5']}})

In [ ]:
ret = generate_feature_by_kmer_loc(dic_seq, need_log=True)

In [13]:
df = pd.DataFrame(ret)

In [15]:
df.head()

,ENSMUST00000034460,ENSMUST00000045312,ENSMUST00000034564,ENSMUST00000045540,ENSMUST00000068519,ENSMUST00000026608,ENSMUST00000036331,ENSMUST00000105894,ENSMUST00000164099,ENSMUST00000029777,...,ENSMUST00000190279,ENSMUST00000129507,ENSMUST00000039174,ENSMUST00000178808,ENSMUST00000039796,ENSMUST00000111326,ENSMUST00000166411,ENSMUST00000113337,ENSMUST00000049031,ENSMUST00000027252
CDS_AAA,0.010698,0.023049,0.021798,0.006540,0.012528,0.013730,0.005752,0.006948,0.010989,0.012376,...,0.008969,0.004975,0.015688,0.009294,0.004261,0.004975,0.050336,0.012921,0.018857,0.037887
CDS_AAT,0.008372,0.015551,0.010899,0.005351,0.003417,0.009153,0.004108,0.005095,0.007212,0.014851,...,0.000000,0.003554,0.010459,0.013011,0.002841,0.007463,0.030201,0.008810,0.016571,0.018083
CDS_AAC,0.010698,0.018884,0.027248,0.009512,0.007973,0.022883,0.014790,0.006948,0.010989,0.016089,...,0.017937,0.007107,0.020113,0.011152,0.011364,0.008706,0.015101,0.012138,0.015429,0.013490
CDS_AAG,0.028372,0.046931,0.019074,0.014863,0.022779,0.038902,0.021364,0.028254,0.023352,0.021040,...,0.026906,0.012082,0.026549,0.026022,0.011364,0.016169,0.057886,0.026821,0.019429,0.058840
CDS_ATA,0.005581,0.006387,0.000000,0.002973,0.007973,0.006865,0.002465,0.005095,0.004121,0.014851,...,0.008969,0.000711,0.003218,0.009294,0.004261,0.006219,0.013423,0.007048,0.009143,0.010620


In [93]:
df.to_parquet('./notebook/feature_intestine.gzip.parquet',compression='gzip')

In [5]:
df= pd.read_parquet('./notebook/feature_intestine.gzip.parquet')

In [158]:
df.fillna(0.0,inplace=True)

In [7]:
df_s1 = pd.read_excel(r'D:/workspace/rnamotif/samples/aan2399_table_S1.xlsx')

In [106]:
df_s1 = df_s1[df_s1.target_id.isin(list(df.columns))]

In [194]:
cols_all = list(df_s1[(df_s1.mean_obs > 3.5) & (df_s1.qval < 0.2)].target_id)
cols = [cols_all[x] for x in random.sample(range(len(cols_all)), 50)]

In [196]:
df_y = {x: float(df_s1[df_s1.target_id == x].b) for x in cols}
df_y = pd.DataFrame(df_y, index=[0])

In [198]:
df_x = df[cols]

In [199]:
num_test = int(len(cols) * 0.1)
col_train = [cols[x] for x in random.sample(range(len(cols)),len(cols)-num_test)]
col_test = [x for x in cols if x not in col_train]

In [257]:
arr_x_train = df[col_train].to_numpy().transpose()
arr_y_train = df_y[col_train].to_numpy()[0]

arr_x_test = df[col_test].to_numpy().transpose()
arr_y_test = df_y[col_test].to_numpy()[0]

In [252]:
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPRegressor

In [264]:
#clf = svm.SVR() # RandomForestClassifier(n_estimators=10)
clf = MLPRegressor(random_state=1, max_iter=400)

In [265]:
clf.fit(arr_x_train, arr_y_train)

MLPRegressor(max_iter=400, random_state=1)

In [266]:
clf.predict(arr_x_test)

array([ 0.16340888,  0.75119252,  0.94141605, -0.04745657,  0.80033402])

In [267]:
arr_y_test

array([ 0.64868073,  1.37995799, -0.9906175 , -0.83406619,  0.54300929])

In [262]:
arr_x_test.shape

(5, 4032)

In [263]:
arr_x_test

array([[0.00431965, 0.01079914, 0.00215983, ..., 0.        , 0.02040816,
        0.        ],
       [0.01578073, 0.0166113 , 0.01453488, ..., 0.        , 0.        ,
        0.        ],
       [0.01318681, 0.01483516, 0.01648352, ..., 0.        , 0.        ,
        0.        ],
       [0.00605327, 0.00786925, 0.01029056, ..., 0.        , 0.        ,
        0.        ],
       [0.01064396, 0.01224055, 0.01756253, ..., 0.        , 0.        ,
        0.        ]])